# 💧 WaterWorks: Glassware Calibration

Using this **COLAB notebook**, we will upload your personal glassware calibration data to later determine corrected delivered volumes. Along the way you will be able to answer Post Lab questions for Lab 3 - WaterWorks: Glassware Calibration.


## Let's get started! 😀

Let's start out by uploading our data into this notebook.

<font color='green'>Start by executing the code below by clicking on the cell and **holding "Shift+Enter"** at the same time.</font>

You should see a button appear with the text "Choose Files".

<font color='green'>**Click on this button** and grab the data (GlasswareCalibrationData.csv) you saved earlier.</font>

In [ ]:
import pandas as pd
import io
import scipy as sc
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
uploaded = files.upload()

Now that the csv file has been uploaded, run the cell below this text which will name your file under the name "data".

<font color='green'>In the cell below, **hold "Shift + Enter"** at the same time. This cell should print all of the data in this file.

Here we will be able to see the column header names which will be important in the next step

<font color='red'>If you have renamed this file, be sure to also **change the name of the file** being called upon as 'GlasswareCalibrationData.csv' below.</font>

In [ ]:
# Python program to read CSV file without header

data = pd.read_csv('GlasswareCalibrationData.csv', encoding= 'unicode_escape')
data = pd.DataFrame(data, index=None)
data=data.tail(-1)
ndata = data.count(axis=0, level=None, numeric_only=False) #count the number of data points in each dataset (column)

#Pipette Volume Calibration

First, we will determine the mass of water delivered as

<font color='blue'>$$\text{mass delivered= final mass - initial mass}$$

In [ ]:
pipette_n_trials=ndata.get(0) #determine the number of trials included in the data file
pipette_initial_mass=np.float_(data['Unnamed: 1']) #finds the column of initial mass
pipette_final_mass=np.float_(data['Unnamed: 2']) #finds the column of final mass
pipette_mass_delivered=np.subtract(pipette_final_mass,pipette_initial_mass) #calculates the mass of water delivered
pipette_mass_delivered=pipette_mass_delivered[~np.isnan(pipette_mass_delivered)] #cleans up the data by removing all NaN cells
pipette_average_mass_delivered=np.average(pipette_mass_delivered)

The following cell defines the function for determining the density (g/mL) of pure water at any temperature (T) in degress Celcius, as a polynomial function, as described in [Kell GS. *J Chera Eng Data.* 1975;20:97](https://pubs.acs.org/doi/pdf/10.1021/je60064a005).


In [ ]:
#This equation will return the density (g/mL) of pure water at any temperature (T) in degrees Celcius. Taken from Kell GS. J Chera Eng Data. 1975;20:97.
def density(T):
  return (999.83951+16.945176*T-7.9870401*10**(-3)*T**2-46.170461*10**(-6)*T**3+105.56302*10**(-9)*T**4-280.54253*10**(-12)*T**5)/(1+16.897850*10**(-3)*T)*0.001

We can calculate the denisty of water at any temperature, by typing `density(T)` and running the cell.
Try this out for a couple of temperatures including those listed in the Post-Lab.

In [ ]:
density(T) #This equation will return the density (g/mL) of pure water at any temperature (T) in degrees Celcius.

The next cell, will calculate the density of the water for each trial using the temperature you input in your Excel sheet.

In [ ]:
pipette_temperature=np.float_(data['Unnamed: 3']) #converting to a float for math operations
pipette_water_density=[] #setting up an empty array
for i in range(0,pipette_n_trials):
  temp=pipette_temperature[i] #grabbing the temperature at trial i
  pipette_calculated_density=density(temp) #calculating the density at the trial temperature
  pipette_water_density.append(pipette_calculated_density) #add this density to the array
pipette_water_density=np.float_(pipette_water_density) #convert to a float for future math operations

Now we have an array with the density of the water at each trial. This will now allow us to convert from mass of water delivered to volume of water delivered.

<font color='blue'> $$\frac{\text{mass delivered}}{\text{density of water}}=\frac{g}{\frac{g}{mL}}=\text{mL of water delivered}$$

In [ ]:
pipette_volume_delivered=np.divide(pipette_mass_delivered, pipette_water_density)
pipette_average_volume_delivered=pipette_volume_delivered.mean()
pipette_volume_range=pipette_volume_delivered.max()-pipette_volume_delivered.min()
print("The average mass of water delivered from your 50 mL pipette is {:3.4f} g.".format(pipette_average_mass_delivered))
print("The average volume delivered from your 50 mL pipette is {:3.4f} mL.".format(pipette_average_volume_delivered))
print("The range of the volumes delivered from your 50 mL pipette is {:3.4f} mL.".format(pipette_volume_range))

#Burette Volume Calibration

First, we will determine the mass of water delivered as

<font color='blue'>$$\text{mass delivered= final mass - initial mass}$$

We will also calculate the expected volume delivered as


<font color='blue'>$$\text{expected volume= final burette reading - initial burette reading}$$

In [ ]:
burette_n_trials=int(ndata.get(5)/5) #determine the number of trials included in the data file
burette_initial_volume=np.float_(data['Unnamed: 7']) #finds the column of initial volumes
burette_final_volume=np.float_(data['Unnamed: 8']) #finds the column of final volumes
burette_expected_volume=np.subtract(burette_final_volume,burette_initial_volume) #calculates the expected volume of water delivered
burette_initial_mass=np.float_(data['Unnamed: 9']) #finds the column of initial mass
burette_final_mass=np.float_(data['Unnamed: 10']) #finds the column of final mass
burette_mass_delivered=np.subtract(burette_final_mass,burette_initial_mass) #calculates the mass of water delivered

The next cell, will calculate the density of the water for each trial using the temperature you input in your Excel sheet.

From there, we can convert from mass of water delivered to volume of water delivered, using the following equation.

<font color='blue'> $$\frac{\text{mass delivered}}{\text{density of water}}=\frac{g}{\frac{g}{mL}}=\text{mL of water delivered}$$

The following cell will also provide us a "correction" value which is defined as

<font color='blue'> $$\text{correction = expected volume - delivered volume}$$

This correction will be calculated for each trial, averaged for each volume range in the burette and the range of these corrections will print below.

In [ ]:
burette_temperature=np.float_(data['Unnamed: 11']) #converting to a float for math operations
burette_water_density=[] #setting up an empty array
for i in range(0,burette_n_trials*5):
  temp=burette_temperature[i] #grabbing the temperature at trial i
  burette_calculated_density=density(temp) #calculating the density at the trial temperature
  burette_water_density.append(burette_calculated_density) #add this density to the array
burette_water_density=np.float_(burette_water_density) #convert to a float for future math operations
burette_volume_delivered=np.divide(burette_mass_delivered, burette_water_density)
burette_correction=np.subtract(burette_expected_volume,burette_volume_delivered)
print("The correction range for the  0-10 mL segment of your burette was {:3.6f}.".format(max(burette_correction[0:3])-min(burette_correction[0:3])))
print("The correction range for the 10-20 mL segment of your burette was {:3.6f}.".format(max(burette_correction[3:6])-min(burette_correction[3:6])))
print("The correction range for the 20-30 mL segment of your burette was {:3.6f}.".format(max(burette_correction[6:9])-min(burette_correction[6:9])))
print("The correction range for the 30-40 mL segment of your burette was {:3.6f}.".format(max(burette_correction[9:12])-min(burette_correction[9:12])))
print("The correction range for the 40-50 mL segment of your burette was {:3.6f}.".format(max(burette_correction[12:15])-min(burette_correction[12:15])))

The following cell will print the average corrections for each burette segment.

In [ ]:
zero_ten_mL_correction= np.mean(burette_correction[0:3])
ten_twenty_mL_correction= np.mean(burette_correction[3:6])
twenty_thirty_mL_correction= np.mean(burette_correction[6:9])
thirty_fourty_mL_correction= np.mean(burette_correction[9:12])
fourty_fifty_mL_correction= np.mean(burette_correction[12:15])
print("zero_ten_mL_correction= {:3.6f}".format(zero_ten_mL_correction))
print("ten_twenty_mL_correction= {:3.6f}".format(ten_twenty_mL_correction))
print("twenty_thirty_mL_correction= {:3.6f}".format(twenty_thirty_mL_correction))
print("thirty_fourty_mL_correction= {:3.6f}".format(thirty_fourty_mL_correction))
print("fourty_fifty_mL_correction= {:3.6f}".format(fourty_fifty_mL_correction))

Now you have the information you need to start working on the next Colab Notebook (BuretteCalibrationCorrections.ipynb).

Copy the print out above that looks something like...

`zero_ten_mL_correction= `

`ten_twenty_mL_correction= `

`twenty_thirty_mL_correction= `

`thirty_fourty_mL_correction= `

`fourty_fifty_mL_correction= `

And paste all of this information where indicated in BuretteCalibrationCorrections.ipynb.